In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import sys
sys.path.append('/glade/u/home/linnia/ctsm6_ppe/')
from utils.pyfunctions import *
utils_path = '/glade/u/home/linnia/ctsm6_ppe/utils/'


2025-08-26 17:06:45.753955: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-26 17:06:45.756741: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-26 17:06:45.763687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756249605.774320    7983 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756249605.777347    7983 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-26 17:06:45.789877: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [15]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
ncores=1
nmem='10GB'
cluster = PBSCluster(
    cores=ncores, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
    account='P93300041', # Input your project ID here
    walltime='5:00:00', # Amount of wall time
    #interface='ib0', # Interface to use
)

# Scale up
cluster.scale(10)

# Setup your client
client = Client(cluster)

### setup

In [2]:
def get_biome_param_names(b, u_params, pft_params):
    
    with open("./biome_configs.pkl", "rb") as f:
        biome_configs = pickle.load(f)

    param_names = list(u_params)
    for pft in biome_configs[b]['pfts']:
        pft_param_names = [f"{param}_{pft}" for param in pft_params]
        param_names.extend(pft_param_names)

    return param_names

In [3]:
#======================== set up ============================
# get parameter information
with open(utils_dir+"/param_names.pkl", "rb") as f:
    param_info = pickle.load(f)
u_params = param_info['u_params']
pft_params = param_info['pft_params']

# get biome information
with open("./biome_configs.pkl", "rb") as f:
    biome_configs = pickle.load(f)

# get default parameter set and reset some settings of default parameters
default_params = pd.read_csv('default_params_norm.csv', index_col=False)
default_params.loc[0, ['jmaxb1']] = [0.4]
default_params.loc[0, ['theta_cj']] = [0.7]
default_params.loc[0, ['upplim_destruct_metamorph']] = [1]
default_params.loc[0, ['xl_12']] = [0]
default_params.loc[0, ['TAU']] = [0.7]
default_params.loc[0, ['RF_SS']] = [0.3]

default_params.loc[0,['leafcn_12']] = [0.9831]
default_params.loc[0,['slatop_12']] = [0.0315]

default_params.loc[0,['leafcn_11']] = [0.9984]
default_params.loc[0,['slatop_11']] = [0.0315]

default_params.loc[0,['KCN_14']] = [0]

# Build universal_set tensor
universal_set = tf.constant(default_params[u_params].iloc[[0]].to_numpy(),dtype=tf.float64)

x_default = tf.constant(default_params.drop(columns=u_params).iloc[[0]].to_numpy(), dtype=tf.float64)


2025-07-31 14:40:57.868837: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [ ]:
# get observations
obs_biome = xr.open_dataset('calibration_obsStatistics_sudokuBiomes.nc')

In [5]:
paths = {
    'lai': './emulators_biomelai_compiled/',
    'gpp': './emulators_biomegpp_compiled/',
    'biomass': './emulators_biomebiomass_compiled/',
}

biomes = [1,2,3,4,5,7,8,9,10,11,12]
param_indices = []
target = []
stdev = []
emulators = []  

for b in biomes:
    biome_name = biome_configs[b]['name']
    param_names = get_biome_param_names(b,u_params,pft_params)
    param_ix = [default_params.columns.get_loc(p)for p in param_names]
    
    param_indices.append(param_ix)
    param_indices.append(param_ix)
    param_indices.append(param_ix)
    
    target.extend([
        tf.convert_to_tensor((obs_biome.LAI_mean).isel(biome=b).values, dtype=tf.float64),
        tf.convert_to_tensor((obs_biome.GPP_mean).isel(biome=b).values, dtype=tf.float64),
        tf.convert_to_tensor((obs_biome.TVC_mean).isel(biome=b).values, dtype=tf.float64)
    ])
    stdev.extend([
        tf.convert_to_tensor((obs_biome.LAI_stdev).isel(biome=b).values, dtype=tf.float64),
        tf.convert_to_tensor((obs_biome.GPP_stdev).isel(biome=b).values, dtype=tf.float64),
        tf.convert_to_tensor((obs_biome.TVC_stdev).isel(biome=b).values, dtype=tf.float64)
    ])
    emulators.extend([
        tf.saved_model.load(f"{paths['lai']}{biome_name}"),
        tf.saved_model.load(f"{paths['gpp']}{biome_name}"),
        tf.saved_model.load(f"{paths['biomass']}{biome_name}")
    ])
    
targets = tf.stack(target, axis=0)
stdevs = tf.stack(stdev, axis=0)


In [6]:
# ——— for fast graph execution ———
lengths = [len(arr) for arr in param_indices]     # list of Python ints

emulator_array = []
for m,input_dim in zip(emulators, lengths):
    sig = tf.TensorSpec([None, input_dim], tf.float64)
    f = tf.function(lambda X, _m=m: _m.compiled_predict_f(X)[0], input_signature=[sig])
    emulator_array.append(f)

In [7]:
# Create a mask for PFT we don't want to calibrate
pft_cols = [c for c in default_params.columns if c not in u_params]
default_pftparams = default_params[pft_cols]
n_params = default_pftparams.shape[1]

mask = np.ones(n_params, dtype=bool)

pfts_dontcal = [10, 12, 14]
for pft in pfts_dontcal:
    pft_param_names = [f"{param}_{pft}" for param in pft_params]
    pft_param_ix = [default_pftparams.columns.get_loc(p) for p in pft_param_names]
    for ix in pft_param_ix:
        mask[ix] = False

trainable_mask_tf = tf.constant(mask, dtype=tf.bool)


### Functions

In [8]:

@tf.function
def optimization_step_batch(x, universal_set, emulator_array, param_indices, optimizer, x_default, barrier_strength=1, lambda_penalty=0.01, trainable_mask=None):
    with tf.GradientTape() as tape:
        batch = tf.shape(x)[0]
        u_tiled = tf.tile(universal_set, [batch, 1])   
        x_full = tf.concat([u_tiled, x], axis=1)
        per_model_losses = []
        
        # loop over metrics and biomes in stacked 1D arrays. 
        for i in range(len(emulator_array)):
            model   = emulator_array[i]

            target = tf.reshape(targets[i], (1, -1))
            stdev = tf.reshape(stdevs[i], (1, -1))
            target_tiled = tf.tile(target, [batch, 1])
            stdev_tiled = tf.tile(stdev, [batch, 1])
        
            ix  = param_indices[i]          
            x_biome = tf.gather(x_full, ix, axis=1)
            y_pred = model(x_biome)
            z = tf.abs((y_pred - target_tiled)/stdev_tiled)

            if epsilon is not None: # if biome/variable zscore is < epsilon, it no longer contributes to loss
                mask = tf.cast(z > epsilon, tf.float64)
                loss_b = tf.reduce_sum(mask * tf.square(z), axis=1)
            else:
                loss_b = tf.reduce_sum(tf.square(z), axis=1)
            
            # size [batch]
            per_model_losses.append(loss_b)
            
        loss_per_sample = tf.add_n(per_model_losses)
        data_loss = tf.reduce_mean(loss_per_sample) # overall scalar loss = mean of the batch
        max_z = tf.sqrt(tf.reduce_max(loss_per_sample)) # worst individual biome/init zscore

        # Penalty for moving from default
        # sum over the parameter‐axis (axis=1) → shape [batch]
        x_default_tiled = tf.tile(x_default, [batch, 1])
        sum_abs = tf.reduce_sum(tf.abs(x - x_default_tiled), axis=1)
        penalty = tf.reduce_mean(sum_abs) # mean of batch

        ratio = penalty / lambda_penalty
        factor = tf.maximum(ratio, 1.0) # if the penalty <=lambda, it no longer impacts loss
        total_loss = data_loss * factor
        
        # penalty for being close to/outside of bounds (0,1)
        barrier = tf.reduce_mean(1.0 / (x + 1e-6) + 1.0 / (1.0 - x + 1e-6))
        total_loss = total_loss * (1.0 + barrier_strength * barrier)
        
    grads = tape.gradient(total_loss, [x])

    if trainable_mask is not None: # mask grad of PFTs we don't want to train
        mask_broadcast = tf.reshape(trainable_mask, (1, -1))
        grads = [tf.where(mask_broadcast, grads, tf.zeros_like(grads[0]))[0]]

    optimizer.apply_gradients(zip(grads, [x]))
    x.assign(tf.clip_by_value(x, 0.0, 1.0))
    return total_loss, max_z, data_loss, factor, barrier_strength*barrier

In [9]:

def run_optimization(x, universal_set, emulator_array, param_indices, x_default, maxiter, lr, lr_decay_steps, checkpoint_N, checkpoint_dir, epsilon, barrier_strength=0, lambda_penalty=0.0,trainable_mask=None):
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr,
        decay_steps=lr_decay_steps,
        decay_rate=0.5,
        staircase=True)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    history_total_loss = []
    history_max_z = []
    history_data_loss = []
    history_penalty = []
    history_barrier = []
    
    for step in range(maxiter):

        total_loss, max_z, data_loss, penalty, barrier_loss = optimization_step_batch(x, universal_set, emulator_array, param_indices, optimizer, x_default, barrier_strength, lambda_penalty,trainable_mask)

        # log
        history_total_loss.append(total_loss.numpy())
        history_max_z.append(max_z.numpy())
        history_data_loss.append(data_loss.numpy())
        history_penalty.append(penalty.numpy())
        history_barrier.append(barrier_loss.numpy())

        # print updates to screen
        if step % 10 == 0:
            tf.print(f"Step {step:03d}: total={total_loss:.6f} max_z={max_z:.6f}")
    
        # Save a checkpoint every N iterations
        if step % checkpoint_N == 0:
            checkpoint = {
                'step': step,
                'params': x,
                'loss': total_loss,
            }
            checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_step_{step}.pkl')
            with open(checkpoint_path, 'wb') as f:
                pickle.dump(checkpoint, f)
                
        # early stopping
        if tf.reduce_max(max_z) <= epsilon:
            print(f"Converged at step {step}")
            tf.print(f"Step {step:03d}: total={total_loss:.6f} max_z={max_z:.6f}")
            break
    
    x_opt = x.numpy()
    logs = {
        'total_loss': history_total_loss,
        'max_z': history_max_z,
        'data_loss': history_data_loss,
        'penalty': history_penalty,
        'barrier': history_barrier
    }
    
    return x_opt, logs

### initialization

In [10]:
# initialization: sample from normal priors around default
pft_cols     = [c for c in default_params.columns if c not in u_params]
default_pftparams = default_params[pft_cols].iloc[0].values   

n_inits = 1000
sigma   = 0.1   # spread of your normal draws

np.random.seed(42)   
x0 = np.random.normal(loc=default_pftparams,
                      scale=sigma,
                      size=(n_inits, default_pftparams.size))

# enforce bounds [0,1]
x0 = np.clip(x0, 0.0, 1.0)

x_init = tf.Variable(
    tf.constant(x0, dtype=tf.float64),
    trainable=True,
    name="x"
)


In [11]:
for pft in pfts_dontcal:
    pft_param_names = [f"{param}_{pft}" for param in pft_params]
    pft_cols     = [c for c in default_params.columns if c not in u_params]
    default_pftparams = default_params[pft_cols]
    pft_param_ix = [default_pftparams.columns.get_loc(p)for p in pft_param_names]
    x0[:,pft_param_ix] = np.tile(default_params[pft_param_names],(n_inits,1))

In [12]:
x_init = tf.Variable(
    tf.constant(x0, dtype=tf.float64),
    trainable=True,
    name="x"
)

### calibration

In [2]:
%%time
# Run optimization
epsilon = 1
lr = 1e-2
maxiter=1000
lr_decay_steps = 100
checkpoint_N = 50
checkpoint_dir = './checkpoints_objective_v3/'
lambda_penalty = 25 # worst sum of delta from default you will tolerate
barrier_strength = 0 # clipping is on

x_opt, logs = run_optimization(x_init, universal_set, emulator_array, param_indices, x_default, 
                               maxiter, lr, lr_decay_steps, checkpoint_N, checkpoint_dir, 
                               epsilon, barrier_strength, lambda_penalty,trainable_mask_tf)


KeyboardInterrupt



In [ ]:
# =========================== save ======================================
n_inits = np.shape(x_opt)[0]
uset_tiled = np.tile(default_params[u_params].values[0], (n_inits,1))
opt_sets = np.concatenate([uset_tiled,x_opt],axis=1)
calibrated_paramsets = pd.DataFrame(opt_sets,columns=default_params.columns.values)

outfile = checkpoint_dir+'calibrated_sets_lambda25_073125.csv'
calibrated_paramsets.to_csv(outfile,index=False)

### plot convergence logs

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(logs['total_loss'], label='Total loss')
plt.plot(logs['data_loss'], label='Data loss')
plt.plot(logs['penalty'], linestyle='--',label='Penalty loss')
#plt.plot(logs['barrier'], label='Barrier loss')
plt.xlabel('Step')
plt.ylabel('Loss value')
plt.legend()
plt.grid(True)
plt.title('Loss components over optimization steps')
plt.ylim(0,100)
plt.show()
